In [20]:
import pandas as pd
import re

# Cargar el archivo TXT inicial
file_path_initial = 'Interactions_Intact__bacillus_subtilis_CW14.txt'
data_initial = pd.read_csv(file_path_initial, delimiter='\t')

# Procesar las columnas de alias
data_initial['Alias(es) interactor A'] = data_initial['Alias(es) interactor A'].str.split('|')
data_initial['Alias(es) interactor B'] = data_initial['Alias(es) interactor B'].str.split('|')

# Función para extraer nombres de genes de un alias
def extract_gene_names(alias_list):
    gene_names = []
    if isinstance(alias_list, list):  # Asegurarse de que alias_list es una lista
        for alias in alias_list:
            # Buscar un nombre de gen que siga el patrón 'uniprotkb:GENE_NAME(gene name)'
            match = re.search(r'uniprotkb:([\w-]+)\(gene name\)', alias)
            if match:
                gene_names.append(match.group(1))
    return ','.join(gene_names)  # Convertir la lista de nombres a una cadena separada por comas

# Asegurarse de que las columnas de alias están en el formato adecuado
data_initial['Alias(es) interactor A'] = data_initial['Alias(es) interactor A'].apply(lambda x: eval(x) if isinstance(x, str) else x)
data_initial['Alias(es) interactor B'] = data_initial['Alias(es) interactor B'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Aplicar la función a las columnas de alias
data_initial['Interactor A Genes'] = data_initial['Alias(es) interactor A'].apply(extract_gene_names)
data_initial['Interactor B Genes'] = data_initial['Alias(es) interactor B'].apply(extract_gene_names)
# Seleccionar solo las columnas de interés
columnas_interes = ['Interactor A Genes', 'Interactor B Genes', 'Host organism(s)', 'Confidence value(s)', 'Taxid interactor A', 'Taxid interactor B']
data_initial = data_initial[columnas_interes]


# Cargar el archivo CSV con los datos filtrados y con Fold Change
file_path_fc = 'deSeqRes_filtrado_bacillus_subtilis_CW14_with_FC.csv'
data_with_fc = pd.read_csv(file_path_fc)

# Renombrar las columnas
data_with_fc = data_with_fc.rename(columns={
    'GeneSymbol': 'Genes',
    'DirectionalFoldChange': 'FoldChange'
})

# Crear una nueva columna 'Regulation' basada en el signo de 'FoldChange'
data_with_fc['Regulation'] = data_with_fc['FoldChange'].apply(lambda x: 'UpRegulated' if x > 0 else 'DownRegulated')

# Tomar el valor absoluto de 'FoldChange'
data_with_fc['FoldChange'] = data_with_fc['FoldChange'].abs()

# Realizar el join en ambas columnas 'Interactor A Genes' y 'Interactor B Genes'
merged_data_A = data_initial_selected.merge(data_with_fc, left_on='Interactor A Genes', right_on='Genes', how='left')
merged_data_B = data_initial_selected.merge(data_with_fc, left_on='Interactor B Genes', right_on='Genes', how='left')

# Concatenar los resultados y eliminar duplicados
merged_data = pd.concat([merged_data_A, merged_data_B]).drop_duplicates()

# Eliminar los registros donde 'Genes' sea NaN
merged_data = merged_data.dropna(subset=['Genes'])

# Filtrar por 'Interaction detection method(s)' solo las filas que sean 'High Throughput'
merged_data = merged_data[merged_data['Interaction detection method(s)'] == 'High Throughput']

# Filtrar por 'Confidence value(s)' solo las filas que sean mayores o iguales a 0.90
merged_data = merged_data[merged_data['Confidence value(s)'].astype(float) >= 0.90]

# Ordenar el DataFrame por 'FoldChange' de mayor a menor
merged_data = merged_data.sort_values(by='FoldChange', ascending=False)

# Agrupar por 'Genes' y combinar los valores en listas, excepto 'FoldChange' y 'Regulation'
grouped_data = merged_data.groupby('Genes').agg({
    'Interactor A Genes': lambda x: list(x),
    'Interactor B Genes': lambda x: list(x),
    'Taxid interactor A': lambda x: list(x),
    'Taxid interactor B': lambda x: list(x),
    'Interaction detection method(s)': lambda x: list(x),
    'Confidence value(s)': lambda x: list(x),
    'FoldChange': 'mean',
    'Regulation': 'first'
}).reset_index()

# Guardar el resultado en un nuevo archivo CSV
#grouped_data.to_csv('Intact_data_bacillus_subtilis.csv', index=False)

# Guardar el resultado en un nuevo archivo Excel
#grouped_data.to_excel('Intact_data_bacillus_subtilis.xlsx', index=False)

In [22]:
data_initial.head()

,Interactor A Genes,Interactor B Genes,Host organism(s),Confidence value(s),Taxid interactor A,Taxid interactor B
0,TNF,TNF,taxid:-1(in vitro)|taxid:-1(In vitro),intact-miscore:0.44,taxid:9606(human)|taxid:9606(Homo sapiens),taxid:9606(human)|taxid:9606(Homo sapiens)
1,,ADM,taxid:-1(in vitro)|taxid:-1(In vitro),intact-miscore:0.4,taxid:9606(human)|taxid:9606(Homo sapiens),taxid:9606(human)|taxid:9606(Homo sapiens)
2,ADM,,taxid:-1(in vitro)|taxid:-1(In vitro),intact-miscore:0.4,taxid:9606(human)|taxid:9606(Homo sapiens),taxid:9606(human)|taxid:9606(Homo sapiens)
3,,CCL2,taxid:-1(in vitro)|taxid:-1(In vitro),intact-miscore:0.44,"taxid:-2(chemical synthesis)|taxid:-2(""Chemica...",taxid:9606(human)|taxid:9606(Homo sapiens)
4,CCL2,,taxid:-1(in vitro)|taxid:-1(In vitro),intact-miscore:0.44,taxid:9606(human)|taxid:9606(Homo sapiens),"taxid:-2(chemical synthesis)|taxid:-2(""Chemica..."
